## Functor (endofunctor)

```
f a -> f b
```

In [ ]:
public class Functor<T>
{
    private readonly T _value;
    
    public Functor(T value)
    {
        _value = value;
    }

    public Functor<T2> Map<T2>(Func<T, T2> fn)
    {
        return new Functor<T2>(fn(_value));
    }
}

### IEnumerable functor

In [ ]:
new List<string>() // Functor constructor/return: F(a)
    .Select( // Map: f(a) -> f(b)
        s => int.Parse(s) // Function: f: a -> b
    );

## Currying

In [ ]:
// F# functions are curried by default

let add a b = a + b
let addMore a b c = a + b + c

let add5 = add 5
let add5_2 = (addMore 5) 2

(add5 2).Display()
(add5_2 7).Display()

7

14

In [ ]:
// Curry2..N
public static Func<TIn2, TOut> Curry<TIn1, TIn2, TOut>(this Func<TIn1, TIn2, TOut> @this, TIn1 value)
{
    return j => @this(value, j);
}

public static Func<TIn2, Func<TIn3, TOut>> Curry<TIn1, TIn2, TIn3, TOut>(this Func<TIn1, TIn2, TIn3, TOut> @this, TIn1 value)
{
    return j => k => @this(value, j, k);
}

In [ ]:
Func<int, int, int> Add = (int a, int b) => a + b;
Func<int, int, int, int> AddMore = (int a, int b, int c) => a + b + c;

var add5 = Add.Curry(5);
var add5_2 = AddMore.Curry(5)(2);

add5(2)
    .Display();
add5_2(7)
    .Display();

7

14

## Composition

In [ ]:
let double n = 2 * n
let subtractOne n = n - 1
let nextOdd = double >> subtractOne

nextOdd 2

3

In [ ]:
// C# compose helper
public static Func<TIn, TOut2> ComposeLeft<TIn, TOut, TOut2>(this Func<TIn, TOut> inner, Func<TOut, TOut2> outer)
{
    return i => outer(inner(i));
}

In [ ]:
Func<int, int> Double = i => 2 * i;
Func<int, int> SubtractOne = i => i - 1;

// C# composition func
Func<int, int> NextOdd_ = i => SubtractOne(Double(i));

// With composition helper
var NextOdd = Double.ComposeLeft(SubtractOne);

NextOdd(2)

3

## Map/Pipelining

In [ ]:
let square a = a * a

3 |> square
square <| 3

9

In [ ]:
public static TIn Tee<TIn>(this TIn @this, Action<TIn> act)
{
    act(@this);
    return @this;
}

public static TIn Tee<TIn, TOut>(this TIn @this, Func<TIn, TOut> act)
{
    act(@this);
    return @this;
}

public static TOut Map<TIn, TOut>(this TIn @this, Func<TIn, TOut> fn)
{
    return fn(@this);
}

In [ ]:
Func<int, int> Square = i => i * i;

3.Map(Square)

9

## State machines

In [ ]:
#r "nuget: OneOf, 3.0.205"

public class PhoneCall
{
    public class State 
    {
        public record Idle(Data.StateData Data);
        public record OffHook(Data.StateData Data);
        public record Ringing(Data.StateData Data);
        public record Connected(Data.StateData Data);
        public record OnHold(Data.StateData Data);
        public record PhoneDestroyed();
        
        public class ConnectedSum : OneOf.OneOfBase<Connected, OnHold>
        {
            public ConnectedSum(OneOf.OneOf<Connected, OnHold> _) : base(_) {}
            public static implicit operator ConnectedSum(Connected _) => new ConnectedSum(_);
            public static implicit operator ConnectedSum(OnHold _) => new ConnectedSum(_);
        }

        public class Sum : OneOf.OneOfBase<Idle, OffHook, Ringing, ConnectedSum, PhoneDestroyed>
        {
            public Sum(OneOf.OneOf<Idle, OffHook, Ringing, ConnectedSum, PhoneDestroyed> _) : base(_) {}
            public static implicit operator Sum(Idle _) => new Sum(_);
            public static implicit operator Sum(OffHook _) => new Sum(_);
            public static implicit operator Sum(Ringing _) => new Sum(_);
            public static implicit operator Sum(ConnectedSum _) => new Sum(_);
            public static implicit operator Sum(PhoneDestroyed _) => new Sum(_);
        }
    }

    public class Input
    {
        public record PhoneNumber(string Value);
        public record ConnectedDateTime(DateTime Value);
        public record CallDuration(TimeSpan Value);
        public record Message(string Value);
    }

    public class Data
    {      
        public record DialedData(Input.PhoneNumber PhoneNumber);
        public record ConnectedData(Input.PhoneNumber PhoneNumber, Input.ConnectedDateTime Connected);
        public record LeftMessageData(Input.PhoneNumber PhoneNumber, Input.ConnectedDateTime Connected, Input.Message Message);
        public class CallData : OneOf.OneOfBase<OneOf.Types.None, DialedData, ConnectedData, LeftMessageData>
        {
            public CallData(OneOf.OneOf<OneOf.Types.None, DialedData, ConnectedData, LeftMessageData> _) : base(_) {}
            public static implicit operator CallData(OneOf.Types.None _) => new CallData(_);
            public static implicit operator CallData(DialedData _) => new CallData(_);
            public static implicit operator CallData(ConnectedData _) => new CallData(_);
            public static implicit operator CallData(LeftMessageData _) => new CallData(_);
        }

        public record StateData(List<CallData> Summary, CallData Current);
    }
    
    public static State.Idle Return() => new State.Idle(new Data.StateData(new List<Data.CallData>(), new OneOf.Types.None()));
    
    public static State.OffHook PickUp(State.Idle idleState) => new State.OffHook(idleState.Data);
    
    public static State.Ringing Dial(State.OffHook offHookState, Input.PhoneNumber phoneNumber) =>
        new State.Ringing(offHookState.Data with { 
            Current = new Data.DialedData(phoneNumber) });
    
    public static State.Connected CallConnected(State.Ringing ringingState) =>
        new State.Connected(ringingState.Data with {
            Current = new Data.ConnectedData(
                ringingState.Data.Current.AsT1.PhoneNumber,
                new Input.ConnectedDateTime(DateTime.Now)) });
    
    public static State.OnHold PlacedOnHold(State.Connected connectedState) => new State.OnHold(connectedState.Data);
    
    public static State.Connected TakenOffHold(State.OnHold onHoldState) => new State.Connected(onHoldState.Data);
    
    public static State.OffHook LeftMessage(State.Connected connectedState, Input.Message message) =>
        CallDisconnected(new State.Connected(connectedState.Data with {
            Current = new Data.LeftMessageData(
                connectedState.Data.Current.AsT2.PhoneNumber,
                connectedState.Data.Current.AsT2.Connected,
                message) }));
    
    public static State.OffHook CallDisconnected(State.Connected connectedState) => new State.OffHook(connectedState.Data);
    
    public static State.PhoneDestroyed PhoneHurledAgainstWall(OneOf.OneOf<State.OffHook, State.Ringing, State.Connected, State.OnHold> states) => new State.PhoneDestroyed();    

    public static State.Sum GetState(State.Sum state) => state;
}

PhoneCall
    .Return()
    .Map(call => PhoneCall.PickUp(call))
    .Map(call => PhoneCall.Dial(call, new PhoneCall.Input.PhoneNumber("+123")))
    .Map(call => PhoneCall.CallConnected(call))
    .Map(call => PhoneCall.LeftMessage(call, new PhoneCall.Input.Message("Please respond.")))
    .Map(call => PhoneCall.GetState(call))
    .Match(
        idle => "Idle",
        offHook => "Off hook",
        ringing => "Ringing",
        connected => "Connected",
        phoneDestroyed => "Phone destroyed"
    )
    .Display();

Installed Packages OneOf, 3.0.205

Off hook

In [ ]:
new Functor<int>(2).Map(i => i * 2).Display();

Submission#3+Functor`1[System.Int32]